In [92]:
import csv

csvfile = open("decode.csv")
insts = csv.DictReader(csvfile)
insts = list(filter(lambda x: x["pat"] != "", insts))
for i in insts:
    i["name"] = i["name"].strip()

In [93]:
def fill_zero(inst):
    fields = ["invalid", "regWe"]
    for field in fields:
        inst[field] = "Y" if int(inst[field] == "1") else "N"

In [94]:
def process_alu(inst):
    # alu_sel1/alu_sel2
    alu_sel1_dict = {"rs1": "REG", "pc": "PC", "zero": "ZERO"}
    alu_sel2_dict = {"rs2": "REG", "imm": "IMM", "zero": "ZERO"}
    inst["src1Type"] = alu_sel1_dict[inst["src1Type"]]
    inst["src2Type"] = alu_sel2_dict[inst["src2Type"]]

    # alu_func
    inst["aluFunc"] = "ALU_" + inst["aluFunc"].upper()

In [95]:
def process_mem(inst):
    # do nothing here
    # mem_len = inst(13,12)
    # load_U  = inst(14)
    pass

In [96]:
def process_fu(inst):
    inst["fuType"] = inst["fuType"].upper()

In [97]:
def process_inst_type(inst):
    inst["instType"] = "IMM_" + inst["instType"].upper()
    # return inst

In [98]:
def process_fu_ops(inst):
    inst["fuOp"] = inst["fuOp"].upper() if inst["fuOp"] != "" else "OP_X"

In [99]:
funcs = [
    # fill zeros
    fill_zero,
    # alu related signals: [alu_sel1, alu_sel2, alu_func, cmp_U]
    process_alu,
    # mem related signals: []
    process_mem,
    # write-back stage: [wb_sel]
    process_fu,
    # fu ops
    process_fu_ops,
    # determine inst_type
    process_inst_type,
]
for i in insts:
    for f in funcs:
        f(i)
    for k, v in i.items():
        i[k] = str(v)
# insts = processed

In [100]:
for i in insts:
    print(i)

{'pat': '??????? ????? ????? ??? ????? 01101 11', 'name': 'lui', 'invalid': 'N', 'regWe': 'Y', 'src1Type': 'ZERO', 'src2Type': 'IMM', 'aluFunc': 'ALU_ADD', 'fuType': 'ALU', 'instType': 'IMM_U', 'fuOp': 'OP_X'}
{'pat': '??????? ????? ????? ??? ????? 00101 11', 'name': 'auipc', 'invalid': 'N', 'regWe': 'Y', 'src1Type': 'PC', 'src2Type': 'IMM', 'aluFunc': 'ALU_ADD', 'fuType': 'ALU', 'instType': 'IMM_U', 'fuOp': 'OP_X'}
{'pat': '??????? ????? ????? ??? ????? 11011 11', 'name': 'jal', 'invalid': 'N', 'regWe': 'Y', 'src1Type': 'PC', 'src2Type': 'IMM', 'aluFunc': 'ALU_ADD', 'fuType': 'BRU', 'instType': 'IMM_J', 'fuOp': 'JUMP'}
{'pat': '??????? ????? ????? ??? ????? 11001 11', 'name': 'jalr', 'invalid': 'N', 'regWe': 'Y', 'src1Type': 'REG', 'src2Type': 'IMM', 'aluFunc': 'ALU_ADD', 'fuType': 'BRU', 'instType': 'IMM_I', 'fuOp': 'JUMP'}
{'pat': '??????? ????? ????? 000 ????? 11000 11', 'name': 'beq', 'invalid': 'N', 'regWe': 'N', 'src1Type': 'REG', 'src2Type': 'REG', 'aluFunc': 'ALU_EQ', 'fuType'

In [101]:
class Signal:
    l: int = 0
    r: int = 0
    name: str = ""


offset = 0
signals = {}
for k, v in insts[0].items():
    if k == "pat" or k == "name":
        continue
    signal = Signal()
    signals[k] = signal
    signal.name = k
    signal.l = offset
    signal.r = offset + len(v) - 1
    offset += len(v)
    print(f"val {k} = UInt({len(v)}.W)")
control_len = offset

val invalid = UInt(1.W)
val regWe = UInt(1.W)
val src1Type = UInt(4.W)
val src2Type = UInt(3.W)
val aluFunc = UInt(7.W)
val fuType = UInt(3.W)
val instType = UInt(5.W)
val fuOp = UInt(4.W)


In [102]:
isa = {}
for i in insts:
    print(f'def {i["name"]:<{10}} = BitPat("b{i["pat"]}")')

def lui        = BitPat("b??????? ????? ????? ??? ????? 01101 11")
def auipc      = BitPat("b??????? ????? ????? ??? ????? 00101 11")
def jal        = BitPat("b??????? ????? ????? ??? ????? 11011 11")
def jalr       = BitPat("b??????? ????? ????? ??? ????? 11001 11")
def beq        = BitPat("b??????? ????? ????? 000 ????? 11000 11")
def bne        = BitPat("b??????? ????? ????? 001 ????? 11000 11")
def blt        = BitPat("b??????? ????? ????? 100 ????? 11000 11")
def bge        = BitPat("b??????? ????? ????? 101 ????? 11000 11")
def bltu       = BitPat("b??????? ????? ????? 110 ????? 11000 11")
def bgeu       = BitPat("b??????? ????? ????? 111 ????? 11000 11")
def lb         = BitPat("b??????? ????? ????? 000 ????? 00000 11")
def lh         = BitPat("b??????? ????? ????? 001 ????? 00000 11")
def lw         = BitPat("b??????? ????? ????? 010 ????? 00000 11")
def lbu        = BitPat("b??????? ????? ????? 100 ????? 00000 11")
def lhu        = BitPat("b??????? ????? ????? 101 ????? 00000 

In [103]:
isa = {}
for i in insts:
    Value = str()
    for k, v in i.items():
        if k == "name" or k == "pat":
            continue
        Value += v + (", " if k != "fuOp" else "")
    print(f'{i["name"]:<{10}} -> List({Value}),')
    i["control"] = Value
    isa[i["name"]] = {"pat": i["pat"], "ctl": Value}

lui        -> List(N, Y, ZERO, IMM, ALU_ADD, ALU, IMM_U, OP_X),
auipc      -> List(N, Y, PC, IMM, ALU_ADD, ALU, IMM_U, OP_X),
jal        -> List(N, Y, PC, IMM, ALU_ADD, BRU, IMM_J, JUMP),
jalr       -> List(N, Y, REG, IMM, ALU_ADD, BRU, IMM_I, JUMP),
beq        -> List(N, N, REG, REG, ALU_EQ, BRU, IMM_B, BRANCH),
bne        -> List(N, N, REG, REG, ALU_NE, BRU, IMM_B, BRANCH),
blt        -> List(N, N, REG, REG, ALU_LT, BRU, IMM_B, BRANCH),
bge        -> List(N, N, REG, REG, ALU_GE, BRU, IMM_B, BRANCH),
bltu       -> List(N, N, REG, REG, ALU_LTU, BRU, IMM_B, BRANCH),
bgeu       -> List(N, N, REG, REG, ALU_GEU, BRU, IMM_B, BRANCH),
lb         -> List(N, Y, REG, IMM, ALU_ADD, MEM, IMM_I, LB),
lh         -> List(N, Y, REG, IMM, ALU_ADD, MEM, IMM_I, LH),
lw         -> List(N, Y, REG, IMM, ALU_ADD, MEM, IMM_I, LW),
lbu        -> List(N, Y, REG, IMM, ALU_ADD, MEM, IMM_I, LBU),
lhu        -> List(N, Y, REG, IMM, ALU_ADD, MEM, IMM_I, LHU),
sb         -> List(N, N, REG, IMM, ALU_ADD, MEM, IMM_S, 

In [104]:
invalid = ["?" for i in range(control_len)]
invalid[signals["invalid"].l] = "1"
invalid = "".join(invalid)
isa["invalid"] = {"pat": "", "ctl": invalid}
print("b" + isa["invalid"]["ctl"])

b1???????????????????????????
